In [1]:
pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.2 MB/s eta 0:00:00


In [7]:
pip install openai pinecone-client sentence-transformers


In [9]:
import openai
import pinecone
from typing import List, Dict

In [2]:
import os
os.environ["PINECONE_API_KEY"] = "your_secret_key"
os.environ["OPENAI_API_KEY"] = "your_secret_key"

In [5]:
OPENAI_API_KEY = "your_secret_key"
PINECONE_API_KEY = "your_secret_key"
PINECONE_ENVIRONMENT = "us-east-1"

In [3]:
from pinecone import Pinecone

pinecone_client = Pinecone(api_key="pcsk_6hggzA_3b6fDDvw8cLMZXoKuCUer5RerbWysfzjQUZQoeg38jVnc4BtWKYFjMXhstwjWvU")
print(pinecone_client.list_indexes().names())


['business-qa-index', 'qa-model', 'qa']


openAI api key


In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from openai import OpenAI

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "qa"

if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-west-2"
        )
    )


index = pc.Index(index_name)
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

documents = [
    "Our business provides software solutions for financial institutions.",
    "We offer cloud-based CRM systems for small businesses.",
    "Our data security platform helps protect against cyber threats."
]


embeddings = model.encode(documents)

upsert_data = [
    (f"doc-{i}", embeddings[i].tolist(), {"text": documents[i]}) for i in range(len(documents))
]
index.upsert(vectors=upsert_data)
print(f"Uploaded {len(documents)} documents to Pinecone.")


def retrieve_documents(query, top_k=3):
    query_embedding = model.encode([query])[0].tolist()
    result = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    retrieved_docs = [
        match['metadata']['text'] for match in result['matches'] if 'metadata' in match
    ]
    return retrieved_docs


def generate_answer(query, retrieved_docs):
    context = "\n".join(retrieved_docs)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Question: {query}\nContext:\n{context}\nAnswer:"}
    ]


    client = OpenAI()


    try:
        stream = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            stream=True,
            temperature=0.7,
            max_tokens=150
        )

        response = ""
        for chunk in stream:
            response += chunk['choices'][0]['delta'].get('content', '') or ""

        return response.strip()

    except Exception as e:
        return f"An error occurred while generating the answer: {str(e)}"


def qa_system(query):
    retrieved_docs = retrieve_documents(query)
    answer = generate_answer(query, retrieved_docs)
    return answer

# Test the QA system
if __name__ == "__main__":
    test_query = "What software solutions does your business provide?"
    print("User Query:", test_query)
    response = qa_system(test_query)
    print("Assistant Response:", response)


Uploaded 3 documents to Pinecone.
User Query: What software solutions does your business provide?
Assistant Response: An error occurred while generating the answer: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


the quota limit is exceeding in openAI, so trying with llama

In [ ]:
pip install llama-index


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install transformers torch


In [ ]:
import pinecone
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os
import time
from typing import List, Dict


pc = pinecone.Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "business-qa-index"
index = pc.Index(index_name)


model_name = "nvidia/Llama-3.1-Nemotron-70B-Instruct-HF"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


def generate_embedding(text: str) -> List[float]:
    return [0.1] * 1536

def upsert_documents(documents: List[Dict[str, str]]):
    vectors = []
    for doc in documents:
        embedding = generate_embedding(doc['text'])
        vectors.append((doc['id'], embedding, {"text": doc['text']}))
    index.upsert(vectors)
    print(f"Upserted {len(documents)} documents into Pinecone.")

def query_pinecone(query: str, top_k: int = 5) -> List[str]:
    query_embedding = generate_embedding(query)
    result = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return [match['metadata']['text'] for match in result['matches']]


def generate_response(context: List[str], query: str) -> str:
    context_text = "\n".join(context)
    prompt = (
        f"Answer the following question using the provided context:\n\n"
        f"Context:\n{context_text}\n\n"
        f"Question: {query}\n\n"
        f"Answer:"
    )


    response = qa_pipeline(prompt, max_length=200, num_return_sequences=1)
    return response[0]['generated_text'].strip()


def qa_bot(query: str) -> str:
    context = query_pinecone(query)
    if not context:
        return "I couldn't find any relevant information to answer your question."

    return generate_response(context, query)


if __name__ == "__main__":
    # Example documents to upsert
    documents = [
        {"id": "doc1", "text": "Our business specializes in eco-friendly products."},
        {"id": "doc2", "text": "We offer free shipping for orders over $50."},
        {"id": "doc3", "text": "Our return policy allows returns within 30 days of purchase."}
    ]
    upsert_documents(documents)

    # Example query
    user_query = "What is your return policy?"
    answer = qa_bot(user_query)
    print(f"Answer: {answer}")


tokenizer_config.json:   0%|          | 0.00/55.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

...

model-00011-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00012-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00013-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00014-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

KeyboardInterrupt: 

the device memory is not sufficient to run the llama model

using sentence transformers model for embedding

In [ ]:
import pinecone
import os
import time
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

In [ ]:
pc = pinecone.Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "qa-model"
index = pc.Index(index_name)

In [ ]:
# Initialize the Sentence-Transformer model for embedding generation
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize the T5 model for response generation (you can replace it with other models like GPT-Neo)
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Function to generate embeddings using Sentence-Transformer model

In [ ]:
def generate_embedding(text: str) -> List[float]:
    embedding = embedding_model.encode(text)
    return embedding.tolist()


Function to upsert documents into Pinecone

In [ ]:
def upsert_documents(documents: List[Dict[str, str]]):
    vectors = []
    for doc in documents:
        embedding = generate_embedding(doc['text'])
        vectors.append((doc['id'], embedding, {"text": doc['text']}))
    index.upsert(vectors)
    print(f"Upserted {len(documents)} documents into Pinecone.")

Function to query Pinecone for relevant documents

In [ ]:
def query_pinecone(query: str, top_k: int = 10) -> List[str]:
    query_embedding = generate_embedding(query)

    print(f"Query Embedding: {query_embedding[:10]}...")  # Printing only the first 10 elements for brevity

    result = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    # Debug: Print the retrieved matches
    print("Top Matches:")
    for match in result['matches']:
        print(match['metadata']['text'])

    return [match['metadata']['text'] for match in result['matches']]

In [12]:
# Function to generate a response using the T5 model (can be replaced by other models like GPT-Neo)
def generate_response(context: List[str], query: str) -> str:
    context_text = "\n".join(context)
    input_text = f"Context: {context_text}\nQuestion: {query}\nAnswer:"

    # Tokenize the input text for T5 model
    inputs = t5_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    # Generate the response
    outputs = t5_model.generate(input_ids=inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)

    # Decode the response
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()

# Main QA function
def qa_bot(query: str) -> str:
    context = query_pinecone(query)
    if not context:
        return "I couldn't find any relevant information to answer your question."
    return generate_response(context, query)

# Example Usage
if __name__ == "__main__":
    # Example documents to upsert
    documents = [
        {"id": "doc1", "text": "Our business specializes in eco-friendly products."},
        {"id": "doc2", "text": "We offer free shipping for orders over $50."},
        {"id": "doc3", "text": "Our return policy allows returns within 30 days of purchase."}
    ]
    upsert_documents(documents)

    # Example query
    user_query = "What is your return policy?"
    answer = qa_bot(user_query)
    print(f"Answer: {answer}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Upserted 3 documents into Pinecone.
Query Embedding: [-0.00747142406180501, 0.04949778690934181, 0.04888179898262024, 0.0038547413423657417, 0.01962166838347912, 0.01839597523212433, 0.04256435111165047, -0.05115160346031189, -0.060795772820711136, 0.01430878322571516]...
Top Matches:
Our return policy allows returns within 30 days of purchase.
Our business specializes in eco-friendly products.
We offer free shipping for orders over $50.
Answer: Our return policy allows returns within 30 days of purchase. We offer free shipping for orders over $50. Question: What is your return policy?


Optimize Embedding Generation

In [15]:
def generate_batch_embeddings(texts: List[str]) -> List[List[float]]:
    embeddings = embedding_model.encode(texts, batch_size=32)
    return embeddings.tolist()

Upserted 3 documents into Pinecone.
Query Embedding: [-0.00747142406180501, 0.04949778690934181, 0.04888179898262024, 0.0038547413423657417, 0.01962166838347912, 0.01839597523212433, 0.04256435111165047, -0.05115160346031189, -0.060795772820711136, 0.01430878322571516]...
Top Matches:
Our return policy allows returns within 30 days of purchase.
Our business specializes in eco-friendly products.
We offer free shipping for orders over $50.
Answer: Our return policy allows returns within 30 days of purchase. We offer free shipping for orders over $50. Question: What is your return policy?


Optimize Pinecone Queries

In [19]:
def query_pinecone(query: str, top_k: int = 5) -> List[str]:
    query_embedding = generate_embedding(query)
    result = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return [match['metadata']['text'] for match in result['matches']]


Upserted 3 documents into Pinecone.
Answer: : We offer free shipping for orders over $50. Our return policy allows returns within 30 days of purchase. Our business is specialized in eco-friendly products. Question: how shipping is done?
